# Pré-tratamento de dados

## Introdução

Este notebook contém pré-tratamento de dados. Elimina colunas que não serão usadas ou que possuam variância zero e índices com valores NaN. Também faz a divisão do dataset em treino e teste. Os dataframes obtidos após tudo isso são guardados na pasta 'dados_tratados'.

## Carrega os dados

O código abaixo lê os dados no arquivo 'earthquake_data_tsunami.csv' obtido no site kaggle.

In [1]:
import pandas as pd

# Carrega os dados do arquivo
df = pd.read_csv('earthquake_data_tsunami.csv')

# Mostra 5 primeiras linhas do dataset
df.head()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.000,-9.7963,159.596,2022,11,1
1,6.9,4,4,735,99,2.229,34.0,25.000,-4.9559,100.738,2022,11,0
2,7.0,3,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,2022,11,1
3,7.3,5,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,2022,11,1
4,6.6,0,2,670,131,4.998,27.0,624.464,-25.5948,178.278,2022,11,1


In [2]:
print(f'O tamanho do df antes do pré-tratamento de dados é: {len(df)}')

O tamanho do df antes do pré-tratamento de dados é: 782


A coluna cdi apresenta valores 0 que indicam a não disponibilidade de informação, portanto, são dados que devem ser removidos. O código abaixo faz uma lista com todos os registros que possuem cdi 0 e contabiliza.

In [3]:
# Define índices com cdi=0
indices = [i for i in df.index if df['cdi'][i]==0]

print(f'O número de índices com cdi igual a zero é: {len(indices)}')

O número de índices com cdi igual a zero é: 212


Como cdi não traz muita informação considerando a presença do mmi, não vale a pena perder 212 registros de terremotos para preservar a coluna de cdi. Ela será removida durante o pré-tratamento de dados.

## Trata os dados

### Retira colunas que não serão usadas

O código abaixo retira as colunas 'Year' e 'Month', pois o objetivo não é estudar padrões temporais, mas sim terremotos isoladamente. Também retira a coluna 'cdi' pelo motivo apresentado anteriormente de que possui 212 registros faltantes. 

In [4]:
# Retira colunas que não serão usadas
df = df.drop(['Year', 'Month', 'cdi'], axis=1)

df.head()

,magnitude,mmi,sig,nst,dmin,gap,depth,latitude,longitude,tsunami
0,7.0,7,768,117,0.509,17.0,14.000,-9.7963,159.596,1
1,6.9,4,735,99,2.229,34.0,25.000,-4.9559,100.738,0
2,7.0,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,1
3,7.3,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,1
4,6.6,2,670,131,4.998,27.0,624.464,-25.5948,178.278,1


Observe que as três colunas não estão mais presentes no DataFrame.

### Retira índices com NaN

O código abaixo retira do DataFrame os índices que possuem valore NaN.

In [5]:
# Remove índices com NaN
df = df.dropna()

### Retira colunas com variância zero

O código abaixo usa a função VarianceThreshold do scikit-learn para retirar colunas com variância zero. 

In [6]:
from sklearn.feature_selection import VarianceThreshold

# Retira colunas com variância zero
seletor = VarianceThreshold()
seletor.fit(df)
df_ = seletor.transform(df) # df vira np.ndarray

# Transforma em DataFrame
df = pd.DataFrame(df_, columns=[*df.columns])

In [7]:
print(f'O tamanho do df após pré-tratamento de dados é: {len(df)}')
print()

O tamanho do df após pré-tratamento de dados é: 782



Não foi perdido nenhum resgistro de terremoto, o que significa que não havia dados com NaN.

In [8]:
df.head()

,magnitude,mmi,sig,nst,dmin,gap,depth,latitude,longitude,tsunami
0,7.0,7.0,768.0,117.0,0.509,17.0,14.000,-9.7963,159.596,1.0
1,6.9,4.0,735.0,99.0,2.229,34.0,25.000,-4.9559,100.738,0.0
2,7.0,3.0,755.0,147.0,3.125,18.0,579.000,-20.0508,-178.346,1.0
3,7.3,5.0,833.0,149.0,1.865,21.0,37.000,-19.2918,-172.129,1.0
4,6.6,2.0,670.0,131.0,4.998,27.0,624.464,-25.5948,178.278,1.0


Não foi perdida nenhuma coluna devido à variância zero.

### Determina colunas de atributos e de target

O código abaixo define colunas de atributos e target.

In [9]:
# Define colunas de atributos e target
TARGET = ['tsunami']
ATRIBUTOS = [*df.columns] # Adiciona todas as colunas
ATRIBUTOS.remove(*TARGET) # Retira colunas target

print(f'Colunas atributos: {ATRIBUTOS}')
print()
print(f'Coluna target: {TARGET}')

Colunas atributos: ['magnitude', 'mmi', 'sig', 'nst', 'dmin', 'gap', 'depth', 'latitude', 'longitude']

Coluna target: ['tsunami']


### Arredonda valores do DataFrame

Valores para arredondamento escolhidos com base no significado de cada coluna e até que precisão seus valores possuem significado útil para o treino dos modelos. Para saber mais sobre o significado de cada coluna, ler o README.

In [10]:
casa_arredondamento = {
    "sig": -1,      # sem unidade
    "nst": -1,      # sem unidade
    "dmin": 2,      # duas casas decimais
    "gap": 0,       # sem decimal
    "depth": -1,    # sem unidade
    "latitude": 2,  # duas casas decimais
    "longitude": 2, # duas casas decimais
}

# Arredondamento
df = df.round(casa_arredondamento)

### Colapsa linhas com valores iguais

In [11]:
# colapsar linhas com valor igual em uma
grouped = df.groupby(ATRIBUTOS, sort=False)
df = grouped.mean().reset_index() # média dos valores para linha resultante

print(f'O tamanho do df após colapso de índices é: {len(df)}')
print()
df.head()

O tamanho do df após colapso de índices é: 782



,magnitude,mmi,sig,nst,dmin,gap,depth,latitude,longitude,tsunami
0,7.0,7.0,770.0,120.0,0.51,17.0,10.0,-9.80,159.60,1.0
1,6.9,4.0,740.0,100.0,2.23,34.0,20.0,-4.96,100.74,0.0
2,7.0,3.0,760.0,150.0,3.12,18.0,580.0,-20.05,-178.35,1.0
3,7.3,5.0,830.0,150.0,1.86,21.0,40.0,-19.29,-172.13,1.0
4,6.6,2.0,670.0,130.0,5.00,27.0,620.0,-25.59,178.28,1.0


Nenhum resgistro de terremoto foi perdido com colapso de valores muito próximos.

### Divisão em treino e teste

In [12]:
from sklearn.model_selection import train_test_split

# Parâmetro para split
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 9

# Define índices de split
indices = df.index
# Define a variável desbalaceada
y = df['tsunami']
indices_treino, indices_teste = train_test_split(
    indices, 
    test_size=TAMANHO_TESTE, 
    stratify=y,
    random_state=SEMENTE_ALEATORIA
)

# Separa o DataFrame em treino e teste
df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

### Separa os DataFrames em dois: atributos e target

In [13]:
# Treino
X_treino = df_treino.reindex(ATRIBUTOS, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)

# Teste
X_teste = df_teste.reindex(ATRIBUTOS, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

### Salva os DataFrames na pasta dados_tratados

Salvar os DataFrames permite usá-los nos outros arquivos do repositório.

In [14]:
X_treino.to_csv('dados_tratados/X_treino.csv', index=False)
y_treino.to_csv('dados_tratados/y_treino.csv', index=False)
X_teste.to_csv('dados_tratados/X_teste.csv', index=False)
y_teste.to_csv('dados_tratados/y_teste.csv', index=False)

## Conclusão

Este notebook fez o pré-tratamento dos dados. Os dados tratados são guardados na pasta 'dados_tratados' com o fim de utilizá-los nos outros notebooks do repositório.

## Referência

[1] *Global Earthquake-Tsunami Risk Assessment Dataset* por Ahmed Mohamed Zaki. Disponível em: https://www.kaggle.com/datasets/ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset